<a href="https://colab.research.google.com/github/AzmPossum/awesome-possum/blob/master/MAKE_A_COPY_OF_THIS_Transcribe_%26_Summarize_Interviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing some libraries that we will need. You will need to run this once per session of the notebook.
!pip install git+https://github.com/openai/whisper.git
!pip install openai
!sudo apt update && sudo apt install ffmpeg

In [ ]:
# This part of the notebooks converts files to mp3. You can skip this part if your file is already mp3.
import subprocess

CurrentFileName = 'example.mp4' # replace this with your current filename
FinalFileName = 'example.mp3' # this will be the filename after the conversion to mp3.

try:
    subprocess.call(['ffmpeg', '-i', f'{CurrentFileName}', f'{FinalFileName}'])

except Exception as e:
    print(e)
    print('Error While Converting Audio')

ch = input('Press Enter to Close')

In [ ]:
# This part of the code sends your file to whisper and outputs a text file with the speech to text transcript.

whisper_output = !whisper "example.mp3" --model large --language English # Replace "example.mp3" with your filename, note this should be the same as the "FinalFileName above"

with open('example.txt', 'w') as f: # Give this the same name as your file in the line above but with .txt ending.
    for line in whisper_output:
        f.write("%s\n" % line)

In [ ]:
# In this part we will turn our text into chunks so that it will be easier to manage by chat gpt when we summarize.
# For shorter text files you might be able to send the entire file in one go.

#Define chunk_size

chunk_size = 8000
file_name = "example.txt" # Replace this with your filename


def split_file(file_name, chunk_size):
    with open(file_name, 'r') as file:
        text = file.read()

    # split the text based on the chunk size
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks

chunked_text = split_file(file_name,chunk_size)

print(chunked_text)


In [ ]:
# This block of code sends the text file to chat GPT and asks chat GPT to summarize the file.
import openai
import time

openai.api_key = "INPUT API KEY HERE" # put your openai api key here.

total_chunks = len(chunked_text)

modified_chunks = []

for i, chunk in enumerate(chunked_text, start=1):
    modified_chunk = f"[START CHUNK {i}/{total_chunks}]\n{chunk}\n[END CHUNK {i}/{total_chunks}]"
    modified_chunks.append(modified_chunk)

# now, `modified_chunks` contains your text chunks with the appropriate start and end markers
chunks_to_send = modified_chunks

# Inside this function are the prompts for chat GPT. You can modify the first prompt but leave the second part as is.
def process_chunk(chunk, is_last):
    if is_last:
        messages=[
            {"role": "system", "content": "Now you have the entire file. Act as an expert in qualitative text analysis. Produce a summary with the main themes from the text and list a couple of examples under each theme of how the user feels."},
            {"role": "user", "content": chunk}
        ]
    else:
        messages=[
            {"role": "system", "content": "Act as a document loader until you load and remember the content of the next text. There might be multiple files, each file is marked by name in the format ###DOCUMENT NAME. I will send you them by chunks. Each chunk start will be noted as [START CHUNK X/TOTAL] and the end of this chunk will be noted as [END CHUNK X/TOTAL] where x is number of current chunk and total is number of all chunks i will send. I will send you multiple messages with chunks, for each message just reply OK [CHUNKx/Total] don't reply anything else, don't explain the text! Let's begin:"},
            {"role": "user", "content": chunk}
        ]

    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages
    )
    return response['choices'][0]['message']['content']

# Now, we process each chunk in turn:
for i, chunk in enumerate(chunks_to_send, start=1):
    is_last = i == len(chunks_to_send)
    result = process_chunk(chunk, is_last)
    print(f"Result for chunk {i}: {result}")

    # Pause for 3 seconds between each request
    time.sleep(3)

